In [1]:
import numpy as np
import sympy as sp

In [2]:
def criar_vetor_de_simbolos(tamanho):
    """
    Cria um vetor (lista) de símbolos SymPy com o tamanho especificado.

    Parameters:
    tamanho (int): O número de símbolos a serem criados.

    Returns:
    list: Lista contendo os símbolos criados.
    """
    # Cria os símbolos usando sp.symbols
    return sp.symbols(f'x1:{tamanho+1}')

In [3]:
# Função para calcular o produto Kronecker considerando binário exclusivo
def kronecker_binary(vectors):
    result = np.array([1])  # Inicia com o elemento neutro da multiplicação
    for v in vectors:
        result = np.kron(result, v)
    return result

In [4]:
def decimal_para_binario_vetor(numero_decimal, tamanho_bits=8):
    """
    Converte um número decimal para um vetor binário usando produtos de Kronecker.

    Args:
        numero_decimal (int): Número decimal a ser convertido.
        tamanho_bits (int, opcional): Número de bits para representar o número binário. Padrão é 8.

    Returns:
        np.ndarray: Vetor resultante da conversão.
    
    Raises:
        ValueError: Se o número decimal for negativo ou se o tamanho de bits for insuficiente.
    """
    # Mapeamento dos caracteres binários para os vetores desejados
    mapeamento = {'0': [1, 0], '1': [0, 1]}

    # Verifica se o número é um inteiro não negativo
    if not isinstance(numero_decimal, int):
        print(numero_decimal)
        raise TypeError("O número decimal deve ser um inteiro.")
    if numero_decimal < 0:
        raise ValueError("A função não suporta números decimais negativos.")

    # Converter o número decimal para binário e remover o prefixo '0b'
    numero_binario = bin(numero_decimal)[2:]

    # Verifica se o número binário cabe no tamanho de bits desejado
    if len(numero_binario) > tamanho_bits:
        raise ValueError(f"O número binário '{numero_binario}' excede o tamanho de bits especificado ({tamanho_bits} bits).")

    # Adiciona zeros à esquerda para completar o tamanho de bits desejado
    numero_binario = numero_binario.zfill(tamanho_bits)

    # Inicializa o vetor com 1 para o produto de Kronecker
    vetor = np.array([1])
    
    # Aplica o produto de Kronecker para cada dígito binário
    for digito in numero_binario:
        vetor = np.kron(vetor, mapeamento[digito])

    return vetor

In [5]:
bits = 4
variaveis = sp.Matrix(kronecker_binary(np.array_split(criar_vetor_de_simbolos(bits*2), bits)))
variaveis

Matrix([
[x1*x3*x5*x7],
[x1*x3*x5*x8],
[x1*x3*x6*x7],
[x1*x3*x6*x8],
[x1*x4*x5*x7],
[x1*x4*x5*x8],
[x1*x4*x6*x7],
[x1*x4*x6*x8],
[x2*x3*x5*x7],
[x2*x3*x5*x8],
[x2*x3*x6*x7],
[x2*x3*x6*x8],
[x2*x4*x5*x7],
[x2*x4*x5*x8],
[x2*x4*x6*x7],
[x2*x4*x6*x8]])

In [6]:
from itertools import product

# Gerar todas as combinações de 0 e 1 para as 4 variáveis
entradas = list(product([0, 1], repeat=4))
entradas

[(0, 0, 0, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 1),
 (0, 1, 0, 0),
 (0, 1, 0, 1),
 (0, 1, 1, 0),
 (0, 1, 1, 1),
 (1, 0, 0, 0),
 (1, 0, 0, 1),
 (1, 0, 1, 0),
 (1, 0, 1, 1),
 (1, 1, 0, 0),
 (1, 1, 0, 1),
 (1, 1, 1, 0),
 (1, 1, 1, 1)]

In [9]:
from itertools import product
import pandas as pd

# Gerar todas as combinações possíveis de 4 bits
entradas = list(product([0, 1], repeat=4))

# Criar as três colunas com base na comparação entre as somas
dados = []
for entrada in entradas:
    soma_a = entrada[0] + entrada[1]
    soma_b = entrada[2] + entrada[3]
    menor = int(soma_a < soma_b)
    igual = int(soma_a == soma_b)
    maior = int(soma_a > soma_b)
    par = (soma_a + soma_b) % 2
    dados.append([menor, igual, maior,par])

# Criar o DataFrame
df = pd.DataFrame(dados, columns=["A < B", "A = B", "A > B", "A + B par"])

# Opcional: adicionar também as entradas no DataFrame, se quiser visualizar
entradas_df = pd.DataFrame(entradas, columns=["A0", "A1", "B0", "B1"])
tabela_verdade = pd.concat([entradas_df, df], axis=1)
tabela_verdade


,A0,A1,B0,B1,A < B,A = B,A > B,A + B par
0,0,0,0,0,0,1,0,0
1,0,0,0,1,1,0,0,1
2,0,0,1,0,1,0,0,1
3,0,0,1,1,1,0,0,0
4,0,1,0,0,0,0,1,1
5,0,1,0,1,0,1,0,0
6,0,1,1,0,0,1,0,0
7,0,1,1,1,1,0,0,1
8,1,0,0,0,0,0,1,1
9,1,0,0,1,0,1,0,0


In [ ]:
def binarios_para_decimais(matriz_binaria):
    """
    Converte uma matriz binária para um vetor de valores decimais.
    
    Parâmetros:
        matriz_binaria (np.ndarray): Matriz 2D de 0s e 1s, onde cada linha representa um número binário.
    
    Retorna:
        np.ndarray: Vetor 1D com os valores decimais de cada linha da matriz.
    """
    num_bits = matriz_binaria.shape[1]
    pesos = 2 ** np.arange(num_bits - 1, -1, -1)
    return matriz_binaria.dot(pesos)

In [26]:
df_saida = tabela_verdade[["A < B", "A = B", "A > B"]]
saida_vetor_dec = binarios_para_decimais(df_saida.to_numpy())
saida_vetor_dec

array([2, 4, 4, 4, 1, 2, 2, 4, 1, 2, 2, 4, 1, 1, 1, 2])

In [24]:
df_saida.to_numpy()

array([[0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0]])

In [27]:
vetores_binarios = []
tamanho_bits = df_saida.columns.size

for numero in saida_vetor_dec:
    try:
        vetor = decimal_para_binario_vetor(int(numero), tamanho_bits)
        vetores_binarios.append(vetor)
    except (TypeError, ValueError) as e:
        print(f"Erro ao converter o número {numero}: {e}")

vetores_binarios = np.array(vetores_binarios).T 
df = pd.DataFrame(vetores_binarios)
vetores_binarios = sp.Matrix(vetores_binarios)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0
2,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
vetores_binarios.shape

(8, 16)

In [29]:
import latexify
from IPython.display import display, Latex

A = sp.Matrix(vetores_binarios)
identity_matrix = sp.eye(A.cols)

In [30]:
@latexify.function(reduce_assignments=True, use_math_symbols=True)
def matrix_column_product(matrix, column_vector):
    """
    Retorna o produto de 'matrix' por 'column_vector'.
    """
    return matrix * column_vector

# Loop pelas colunas da identidade
for i in range(A.cols):
    print(i)
    result_expr = matrix_column_product(A, identity_matrix.col(i))
    pos_1 = result_expr.tolist().index([1])
    # Monta e exibe a equação completa em LaTeX
    display(Latex(
        rf"$$ {sp.latex(A)} \;\cdot\; {sp.latex(identity_matrix.col(i))}"
        rf" = {sp.latex(result_expr)} = {sp.latex(pos_1)} $$"
    ))

0


<IPython.core.display.Latex object>

1


<IPython.core.display.Latex object>

2


...

<IPython.core.display.Latex object>

14


<IPython.core.display.Latex object>

15


<IPython.core.display.Latex object>

In [31]:
regra = sp.Matrix(np.dot(vetores_binarios, variaveis)) # expressões simbólicas
regra

Matrix([
[                                                                                0],
[              x1*x4*x5*x7 + x2*x3*x5*x7 + x2*x4*x5*x7 + x2*x4*x5*x8 + x2*x4*x6*x7],
[x1*x3*x5*x7 + x1*x4*x5*x8 + x1*x4*x6*x7 + x2*x3*x5*x8 + x2*x3*x6*x7 + x2*x4*x6*x8],
[                                                                                0],
[              x1*x3*x5*x8 + x1*x3*x6*x7 + x1*x3*x6*x8 + x1*x4*x6*x8 + x2*x3*x6*x8],
[                                                                                0],
[                                                                                0],
[                                                                                0]])

In [32]:
def criar_substituicoes(bits):
    simb = criar_vetor_de_simbolos(bits * 2)
    bools = sp.symbols(f'b1:{bits+1}')
    subs = {}
    for i in range(bits):
        zero_sym, one_sym = simb[2*i], simb[2*i+1]
        b = bools[i]
        subs[ zero_sym ] = 1 - b   # coordenada “0” → 1-b
        subs[ one_sym  ] = b       # coordenada “1” → b
    return subs


In [33]:
print(bits)
subs = criar_substituicoes(bits)
sp.pprint(subs)

4
{x₁: 1 - b₁, x₂: b₁, x₃: 1 - b₂, x₄: b₂, x₅: 1 - b₃, x₆: b₃, x₇: 1 - b₄, x₈: b₄}


In [34]:
f_booleana = sp.simplify(regra.subs(subs))
f_booleana

Matrix([
[                                                                                                                                                        0],
[                         -b1*b2*b3*(b4 - 1) - b1*b2*b4*(b3 - 1) + b1*b2*(b3 - 1)*(b4 - 1) - b1*(b2 - 1)*(b3 - 1)*(b4 - 1) - b2*(b1 - 1)*(b3 - 1)*(b4 - 1)],
[b1*b2*b3*b4 + b1*b3*(b2 - 1)*(b4 - 1) + b1*b4*(b2 - 1)*(b3 - 1) + b2*b3*(b1 - 1)*(b4 - 1) + b2*b4*(b1 - 1)*(b3 - 1) + (b1 - 1)*(b2 - 1)*(b3 - 1)*(b4 - 1)],
[                                                                                                                                                        0],
[                         -b1*b3*b4*(b2 - 1) - b2*b3*b4*(b1 - 1) + b3*b4*(b1 - 1)*(b2 - 1) - b3*(b1 - 1)*(b2 - 1)*(b4 - 1) - b4*(b1 - 1)*(b2 - 1)*(b3 - 1)],
[                                                                                                                                                        0],
[                                                

In [35]:
from itertools import product

# Gerar todas as combinações de 0 e 1 para as 4 variáveis
entradas = list(product([0, 1], repeat=4))
entradas

[(0, 0, 0, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 1),
 (0, 1, 0, 0),
 (0, 1, 0, 1),
 (0, 1, 1, 0),
 (0, 1, 1, 1),
 (1, 0, 0, 0),
 (1, 0, 0, 1),
 (1, 0, 1, 0),
 (1, 0, 1, 1),
 (1, 1, 0, 0),
 (1, 1, 0, 1),
 (1, 1, 1, 0),
 (1, 1, 1, 1)]

In [36]:
for i, e in enumerate(f_booleana):
    # Ordena os símbolos por nome para manter consistência
    variaveis_ordenadas = sorted(e.free_symbols, key=lambda s: s.name)
    
    # Exibe a forma simbólica da função
    nome_funcao = f"F_{{{i+1}}}({', '.join(str(s) for s in variaveis_ordenadas)})"
    
    bin_valor = []

    for linha_vals in entradas:
        # Mapeia os valores de entrada para os símbolos
        subs_dict = {var: linha_vals[j] for j, var in enumerate(variaveis_ordenadas)}
        entrada_str = ",".join(str(v) for v in linha_vals)
        
        # Constrói a expressão em LaTeX com substituição literal
        expr_latex = sp.latex(e)
        for var in variaveis_ordenadas:
            expr_latex = expr_latex.replace(sp.latex(var), f"({subs_dict[var]})")
        
        # Calcula o valor real da expressão
        resultado = e.subs(subs_dict)
        bin_valor.append(resultado)
   
    # Verificando se teste é igual a alguma coluna
    for coluna in tabela_verdade.columns:
        if tabela_verdade[coluna].tolist() == bin_valor:
            display(Latex(rf"$$ \text{{Coluna }} {i} \text{{ é igual a }} {nome_funcao} = {sp.latex(e)} $$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [37]:
for i, e in enumerate(f_booleana):
    variaveis_ordenadas = sorted(e.free_symbols, key=lambda s: s.name)
    print(f"F{i+1}({", ".join(str(s) for s in variaveis_ordenadas)}) = {e}")
    print("-------------------------")
    bin_valor = []
    for vals in entradas:
        valor = e.subs({var: vals[i] for i, var in enumerate(variaveis_ordenadas)})
        bin_valor.append(valor)
        print(f"{vals[0]} {vals[1]} {vals[2]} {vals[3]} | {valor}")
    
    # Verificando se teste é igual a alguma coluna
    for coluna in tabela_verdade.columns:
        if tabela_verdade[coluna].tolist() == bin_valor:
            print(f"Coluna '{coluna}' é igual a",e)
    else:
        print("Nenhuma coluna é igual a",e)

F1() = 0
-------------------------
0 0 0 0 | 0
0 0 0 1 | 0
0 0 1 0 | 0
0 0 1 1 | 0
0 1 0 0 | 0
0 1 0 1 | 0
0 1 1 0 | 0
0 1 1 1 | 0
1 0 0 0 | 0
1 0 0 1 | 0
1 0 1 0 | 0
1 0 1 1 | 0
1 1 0 0 | 0
1 1 0 1 | 0
1 1 1 0 | 0
1 1 1 1 | 0
Nenhuma coluna é igual a 0
F2(b1, b2, b3, b4) = -b1*b2*b3*(b4 - 1) - b1*b2*b4*(b3 - 1) + b1*b2*(b3 - 1)*(b4 - 1) - b1*(b2 - 1)*(b3 - 1)*(b4 - 1) - b2*(b1 - 1)*(b3 - 1)*(b4 - 1)
-------------------------
0 0 0 0 | 0
0 0 0 1 | 0
0 0 1 0 | 0
0 0 1 1 | 0
0 1 0 0 | 1
0 1 0 1 | 0
0 1 1 0 | 0
0 1 1 1 | 0
1 0 0 0 | 1
1 0 0 1 | 0
1 0 1 0 | 0
1 0 1 1 | 0
1 1 0 0 | 1
1 1 0 1 | 1
1 1 1 0 | 1
1 1 1 1 | 0
Coluna 'A > B' é igual a -b1*b2*b3*(b4 - 1) - b1*b2*b4*(b3 - 1) + b1*b2*(b3 - 1)*(b4 - 1) - b1*(b2 - 1)*(b3 - 1)*(b4 - 1) - b2*(b1 - 1)*(b3 - 1)*(b4 - 1)
Nenhuma coluna é igual a -b1*b2*b3*(b4 - 1) - b1*b2*b4*(b3 - 1) + b1*b2*(b3 - 1)*(b4 - 1) - b1*(b2 - 1)*(b3 - 1)*(b4 - 1) - b2*(b1 - 1)*(b3 - 1)*(b4 - 1)
F3(b1, b2, b3, b4) = b1*b2*b3*b4 + b1*b3*(b2 - 1)*(b4 - 1) + b1*b4*(b2